# Classes and Methods Advanced

## TTable of Contents

- [1. Introduction](#1.-Introduction)
- [2. Positional arguments vs keyword arguments](#2.-Positional-arguments-vs-keyword-arguments)
- [3. Invariants](#3.-Invariants)
- [4. More Advanced __init__ Method](#4.-More-Advanced-__init__-Method)
- [5. Operator Overloading](#5.-Operator-Overloading)
- [6. Type-based Dispatch](#6.-Type-based-Dispatch)
- [7. Polymorphism](#7.-Polymorphism)
- [8. Interface and Implementation](#8.-Interface-and-Implementation)
- [9. Summary](#9.-Summary)


## 1. Introduction

We will continue with defining methods in classes. We will focus on more advanced object-oriented features related to methods. We will also show how to ensure that arguments of methods have correct values. 

## 2. Positional arguments vs keyword arguments

Additionally, arguments when calling a function or method can be either **positional arguments** or **keyword arguments**.
- A *positional argument* does not have a parameter name. It is called based on the order the parameters were defined for such function or method.
- A *keyword argument* uses parameter name (e.g. `param_name=<value>`). It does not rely on the order of the parameters definition.

Let us see an example with the `print_time` function.

In [ ]:
def print_time(hour: int, minute: int, second: int) -> None:
    """
    Prints the hour, minute and time received as parameters for debug 
    purposes.
    :param hour: integer representing an hour
    :param minute: integer representing a minute
    :param second: integer representing a second
    """
    print(f'{hour=:0>2} {minute=:0>2} {second=:0>2}')

<div class="alert alert-info">
    <b>Formatting f-strings</b><br>
    Notice that in the previous example we are using f-strings to format the message. On the one hand, each variable within the curly braces <code>{}</code> is succeeded by the character <code>=</code>. This character prints the name of the variable, the <code>=</code> symbol, and tha value stored in the variable (e.g. <code>hour=10</code>). <br>
    On the other hand, the expression <code>0>2</code> comes after the colon (<code>:</code>), and reads as "show two digits and if the value only has one, add one zero at the beginning". <br>
    If you want to know more about f-strings formatting, please visit the <a href="https://peps.python.org/pep-0498/">official documentation</a>.
</div>

In the following cell, we make use of **only positional arguments**.
We want to print the time "10:05:02".

In [ ]:
print_time(10, 5, 2)

We can also make use of **only keyword arguments**, as follows.

In [ ]:
print_time(hour=10, minute=5, second=2)

The advantage of using **keyword arguments** is that we can also **swap the order of the arguments**!

In [ ]:
print_time(second=2, hour=10, minute=5)

We can also **combine positional and keyword arguments** in the same call.
For instance, in the next cell, we pass the hours as a positional argument, and the rest as keyword arguments.
Notice that the keyword arguments do not follow the original order of the parameters definition!

In [ ]:
print_time(10, second=2, minute=5)

But, beware! If you combine both positional and keyword arguments, ensure the **first arguments are always positional and follow the parameter definition order**, otherwise you will get an error.

In [ ]:
print_time(5, second=2, hour=10)

## 3. Invariants

A `Time` object is well-formed if the values of `minute` and `second` are between 0 and 60
(including 0 but not 60) and if the value of `hour` is between 0 and 24.
Requirements like these are called **invariants** because they should always be true. 
To put it a different way, if they are not true, something has gone wrong.

Writing code to check invariants can help detect errors and find their causes. 
The next cell contains the function `valid_time` that takes a `Time` object and returns `False` if it
violates an **invariant**.

In [ ]:
class Time:
    """
    Represents the time of day.
    attributes: hour, minute, second
    """
    
    def __init__(self, hour: int = 0, minute: int = 0, second: int = 0) -> None:
        """ 
        Initializes a Time object.
        :param hour: integer value for hour
        :param minute: integer value for minute
        :param second: integer value for second
        """
        self.hour: int = hour
        self.minute: int = minute
        self.second: int = second
            
    def __str__(self) -> str:
        """ 
        Returns a string representation of the Time object.
        :returns: string representation of the Time object.
        """
        return '{:02d}:{:02d}:{:02d}'.format(self.hour, self.minute, self.second)
      
#...

    def time_to_int(self) -> int:
        """ 
        Converts a Time object into an integer value representing seconds.
        :param time: a Time object to be converted
        :returns: number of seconds as an integer value.
        """
        minutes: int = 60 * self.hour + self.minute
        seconds: int = 60 * minutes + self.second
        return seconds
    
    def increment(self, seconds: int) -> 'Time':
        """ 
        Adds seconds to a Time object.
        :param time: a Time object
        :param seconds: number of seconds to be added
        :returns: a Time object increased by seconds.
        """
        seconds += self.time_to_int()
        return Time.int_to_time(seconds)
    
    def is_after(self, other) -> bool:
        """ 
        Returns a Boolean to indicate whether the current object is greater 
        than another one.
        :param other: another Time object
        :returns: `True` if the current object is greater than time2, 
            `False` otherwise.
        """
        return self.time_to_int() > other.time_to_int()
    
    def valid_time(self) -> bool:
        """ 
        Checks whether we are dealing with a correct Time object. That is,
        minutes and seconds are between [0, 60) and hours are between [0, 24).
        :param time: Time object to be checked
        :returns: `True` if the time is valid, `False` otherwise.
        """
        valid = True
    
        if self.hour < 0 or self.minute < 0 or self.second < 0:
            valid = False
        if self.hour >= 24 or self.minute >= 60 or self.second >= 60:
            valid = False
        
        return valid

    @staticmethod
    def int_to_time(seconds: int) -> 'Time':
        """ 
        Converts an integer value representing seconds into a Time object.
        :param seconds: as Integer value
        :returns: a Time object.
        """
        time: Time = Time()
        minutes, time.second = divmod(seconds, 60)
        hours, time.minute = divmod(minutes, 60)
        _, time.hour = divmod(hours, 24)
    
        return time

At the beginning of each function you could check the arguments to make sure they are
valid.

In [ ]:
def add_time(t1: Time, t2: Time) -> Time:
    """ 
    Returns a new Time object containing the sum of two Time objects.
    :param time1: a Time object
    :param time2: a Time object
    :returns: a Time object containing the sum of two Time objects.
    """
    if not t1.valid_time() or not t2.valid_time():
        raise ValueError('Invalid Time object in add_time')
        
    seconds: int = t1.time_to_int() + t2.time_to_int()
    return Time.int_to_time(seconds)

Or you could use an **assert statement**, which checks a given invariant and raises an exception
if it fails.
This alternative is useful during the development process but not necessarily to inform your users about a mistake on the use of your program.

In [ ]:
def add_time(t1: Time, t2: Time) -> Time:
    """ 
    Returns a new Time object containing the sum of two Time objects.
    :param time1: a Time object
    :param time2: a Time object
    :returns: a Time object containing the sum of two Time objects.
    """
    assert t1.valid_time() and t2.valid_time(), 'Invalid Time object in add_time'
    
    seconds: int = t1.time_to_int() + t2.time_to_int()
    return Time.int_to_time(seconds)

## 4. More Advanced `__init__` Method

Consider the following cell which contains a rather strange initialisation of an object `Time`.

In [ ]:
start: Time = Time()
print(start)

end: Time = Time(25, 43, 51)
print(end)

As you can see, this results in an invalid time representation!

Here are two alternative implementations for `__init__`.

In [ ]:
class Time:
    """
    Represents the time of day.
    attributes: hour, minute, second
    """
    
    def __init__(self, hour: int = 0, minute: int = 0, second: int = 0) -> None:
        """ 
        Initializes a Time object if valid values are provided. That is,
        0 <= hour < 24, 0 <= minute < 60, and 0 <= second < 60.
        :param hour: integer value for hour
        :param minute: integer value for minute
        :param second: integer value for second
        """
        if (0 <= hour < 24 and 0 <= minute < 60 and 0 <= second < 60):
            self.hour: int = hour
            self.minute: int = minute
            self.second: int = second
        else:
            raise ValueError('Trying to create an invalid time representation!')
            
    def __str__(self) -> str:
        """ 
        Returns a string representation of the Time object.
        :returns: string representation of the Time object.
        """
        return '{:02d}:{:02d}:{:02d}'.format(self.hour, self.minute, self.second)
        
#...
        
    def time_to_int(self) -> int:
        """ 
        Converts a Time object into an integer value representing seconds.
        :param time: a Time object to be converted
        :returns: number of seconds as an integer value.
        """
        minutes: int = 60 * self.hour + self.minute
        seconds: int = 60 * minutes + self.second
        return seconds
    
    def increment(self, seconds: int) -> 'Time':
        """ 
        Adds seconds to a Time object.
        :param time: a Time object
        :param seconds: number of seconds to be added
        :returns: a Time object increased by seconds.
        """
        seconds += self.time_to_int()
        return Time.int_to_time(seconds)
    
    def is_after(self, other) -> bool:
        """ 
        Returns a Boolean to indicate whether the current object is greater 
        than another one.
        :param other: another Time object
        :returns: `True` if the current object is greater than time2, 
            `False` otherwise.
        """
        return self.time_to_int() > other.time_to_int()

    @staticmethod
    def int_to_time(seconds: int) -> 'Time':
        """ 
        Converts an integer value representing seconds into a Time object.
        :param seconds: as Integer value
        :returns: a Time object.
        """
        time: Time = Time()
        minutes, time.second = divmod(seconds, 60)
        hours, time.minute = divmod(minutes, 60)
        _, time.hour = divmod(hours, 24)
    
        return time

Or:

In [ ]:
class Time:
    """
    Represents the time of day.
    attributes: hour, minute, second
    """
    
    def __init__(self, hour: int = 0, minute: int = 0, second: int = 0) -> None:
        """ 
        Initializes a Time object. It ensures that a valid time is created. 
        That is, 0 <= hour < 24, 0 <= minute < 60, and 0 <= second < 60.
        :param hour: integer value for hour
        :param minute: integer value for minute
        :param second: integer value for second
        """
        (ignore, self.hour) = divmod(hour, 24)
        (ignore, self.minute) = divmod(minute, 60)
        (ignore, self.second) = divmod(second, 60)
            
    def __str__(self) -> str:
        """ 
        Returns a string representation of the Time object.
        :returns: string representation of the Time object.
        """
        return '{:02d}:{:02d}:{:02d}'.format(self.hour, self.minute, self.second)

#...

    def time_to_int(self) -> int:
        """ 
        Converts a Time object into an integer value representing seconds.
        :param time: a Time object to be converted
        :returns: number of seconds as an integer value.
        """
        minutes: int = 60 * self.hour + self.minute
        seconds: int = 60 * minutes + self.second
        return seconds
    
    def increment(self, seconds: int) -> 'Time':
        """ 
        Adds seconds to a Time object.
        :param time: a Time object
        :param seconds: number of seconds to be added
        :returns: a Time object increased by seconds.
        """
        seconds += self.time_to_int()
        return Time.int_to_time(seconds)
    
    def is_after(self, other) -> bool:
        """ 
        Returns a Boolean to indicate whether the current object is greater 
        than another one.
        :param other: another Time object
        :returns: `True` if the current object is greater than time2, 
            `False` otherwise.
        """
        return self.time_to_int() > other.time_to_int()

    @staticmethod
    def int_to_time(seconds: int) -> 'Time':
        """ 
        Converts an integer value representing seconds into a Time object.
        :param seconds: as Integer value
        :returns: a Time object.
        """
        time: Time = Time()
        minutes, time.second = divmod(seconds, 60)
        hours, time.minute = divmod(minutes, 60)
        _, time.hour = divmod(hours, 24)
    
        return time

In [ ]:
start: Time = Time()
print(start)

end: Time = Time(25, 43, 51)
print(end)

Given that we defined **default values** for the parameters, if you provide only one argument, it will override the default value of `hour`.

In [ ]:
start: Time = Time(9)
print(start)

If you provide two arguments, it overrides the default values of `hour` and `minutes`.

In [ ]:
start: Time = Time(9, 45)
print(start)

If you provide all three arguments, they override all three default values.

<div class="alert alert-success">
    <b>Do It Yourself!</b><br>
    Do you remember the <i>Dog</i> class? Well, let's define it again but this time you should provide a constructor. The attributes of a dog are its name and its breed.
</div>

In [ ]:
# Remove this line and add your code here

## 5. Operator Overloading

By defining other special methods, you can specify the behavior of operators on
programmer-defined types. 

For example, if you define a method named `__add__` for the
`Time` class, you can use the `+` operator on `Time` objects.

In [ ]:
class Time:
    """
    Represents the time of day.
    attributes: hour, minute, second
    """
    
    def __init__(self, hour: int = 0, minute: int = 0, second: int = 0) -> None:
        """ 
        Initializes a Time object. It ensures that a valid time is created. 
        That is, 0 <= hour < 24, 0 <= minute < 60, and 0 <= second < 60.
        :param hour: integer value for hour
        :param minute: integer value for minute
        :param second: integer value for second
        """
        (ignore, self.hour) = divmod(hour, 24)
        (ignore, self.minute) = divmod(minute, 60)
        (ignore, self.second) = divmod(second, 60)
            
    def __str__(self) -> str:
        """ 
        Returns a string representation of the Time object.
        :returns: string representation of the Time object.
        """
        return '{:02d}:{:02d}:{:02d}'.format(self.hour, self.minute, self.second)

#...    
    
    def time_to_int(self) -> int:
        """ 
        Converts a Time object into an integer value representing seconds.
        :param time: a Time object to be converted
        :returns: number of seconds as an integer value.
        """
        minutes: int = 60 * self.hour + self.minute
        seconds: int = 60 * minutes + self.second
        return seconds
    
    def increment(self, seconds: int) -> 'Time':
        """ 
        Adds seconds to a Time object.
        :param time: a Time object
        :param seconds: number of seconds to be added
        :returns: a Time object increased by seconds.
        """
        seconds += self.time_to_int()
        return Time.int_to_time(seconds)
    
    def is_after(self, other) -> bool:
        """ 
        Returns a Boolean to indicate whether the current object is greater 
        than another one.
        :param other: another Time object
        :returns: `True` if the current object is greater than time2, 
            `False` otherwise.
        """
        return self.time_to_int() > other.time_to_int()

    def __add__(self, other) -> 'Time':
        """ 
        Adds a Time object to the current Time object.
        :param other: the Time object to be added to the current one
        :returns: a Time object.
        """
        seconds: int = self.time_to_int() + other.time_to_int()
        return Time.int_to_time(seconds)
    
    @staticmethod
    def int_to_time(seconds: int) -> 'Time':
        """ 
        Converts an integer value representing seconds into a Time object.
        :param seconds: as Integer value
        :returns: a Time object.
        """
        time: Time = Time()
        minutes, time.second = divmod(seconds, 60)
        hours, time.minute = divmod(minutes, 60)
        _, time.hour = divmod(hours, 24)
    
        return time

In [ ]:
start: Time = Time(9, 45, 30)
duration: Time = Time(0, 15, 30)
print(start + duration)

When you apply the `+` operator to `Time` objects, Python invokes the `__add__` method under the hood.
Something similar happens when you print a value; Python invokes the `__str__` method. 

Changing the behavior of an operator so that it works with programmer-defined types is
called **operator overloading**. 
For every operator (e.g. `<`, `==`, `>=`, `*`) in Python there is a corresponding special method, like `__add__`. 
For more details, see [the official documentation](http://docs.python.org/3/reference/datamodel.html#specialnames).

<div class="alert alert-success">
    <b>Do It Yourself!</b><br>
    Overload the &gt; operator for the <i>Time</i> class. Use the definition provided for the method <i>is_after</i>.
</div>

In [ ]:
# Remove this line and add your code here

## 6. Type-based Dispatch

The `__add__` method introduced in the previous section, adds two `Time` objects. 
It would be convenient to use the `__add__` method to add an integer value (representing a number of seconds) to a `Time` object.

The following is a version of `__add__` that checks the type of the parameter
`other` and invokes either `add_time` or `increment`.

In [ ]:
class Time:
    """
    Represents the time of day.
    attributes: hour, minute, second
    """
    
    def __init__(self, hour: int = 0, minute: int = 0, second: int = 0) -> None:
        """ 
        Initializes a Time object. It ensures that a valid time is created. 
        That is, 0 <= hour < 24, 0 <= minute < 60, and 0 <= second < 60.
        :param hour: integer value for hour
        :param minute: integer value for minute
        :param second: integer value for second
        """
        (ignore, self.hour) = divmod(hour, 24)
        (ignore, self.minute) = divmod(minute, 60)
        (ignore, self.second) = divmod(second, 60)
            
    def __str__(self) -> str:
        """ 
        Returns a string representation of the Time object.
        :returns: string representation of the Time object.
        """
        return '{:02d}:{:02d}:{:02d}'.format(self.hour, self.minute, self.second)

#...

    def time_to_int(self) -> int:
        """ 
        Converts a Time object into an integer value representing seconds.
        :param time: a Time object to be converted
        :returns: number of seconds as an integer value.
        """
        minutes: int = 60 * self.hour + self.minute
        seconds: int = 60 * minutes + self.second
        return seconds
    
    def increment(self, seconds: int) -> 'Time':
        """ 
        Adds seconds to a Time object.
        :param time: a Time object
        :param seconds: number of seconds to be added
        :returns: a Time object increased by seconds.
        """
        seconds += self.time_to_int()
        return Time.int_to_time(seconds)
    
    def is_after(self, other) -> bool:
        """ 
        Returns a Boolean to indicate whether the current object is greater 
        than another one.
        :param other: another Time object
        :returns: `True` if the current object is greater than time2, 
            `False` otherwise.
        """
        return self.time_to_int() > other.time_to_int()

    def add_time(self, other) -> 'Time':
        """ 
        Adds a Time object to the current Time object.
        :param other: the Time object to be added to the current one
        :returns: a Time object.
        """
        seconds: int = self.time_to_int() + other.time_to_int()
        return Time.int_to_time(seconds)
    
    def __add__(self, other : any) -> 'Time':
        """ 
        Adds a Time object or a number of seconds to the current Time object.
        :param other: the Time object or number of seconds to be added 
            to the current one.
        :returns: a Time object.
        """
        if isinstance(other, Time):
            return self.add_time(other)
        else:
            return self.increment(other)
    
    @staticmethod
    def int_to_time(seconds: int) -> 'Time':
        """ 
        Converts an integer value representing seconds into a Time object.
        :param seconds: as Integer value
        :returns: a Time object.
        """
        time: Time = Time()
        minutes, time.second = divmod(seconds, 60)
        hours, time.minute = divmod(minutes, 60)
        _, time.hour = divmod(hours, 24)
    
        return time

The type-hint of the second argument of the method `__add__` is `any`. The reason is that we do not know in advance what the exact type will be, either an Time object or an integer value.

- The built-in function `isinstance` takes a value and a class object, and returns `True` if the value is an instance of the class.
- If `other` is a `Time` object, `__add__` calls `add_time`. 
- Otherwise, it assumes that the parameter is a number and calls `increment`.

This operation is called a **type-based dispatch** because it dispatches the computation to different methods based on the type of the arguments.
In object-oriented languages this is a common concept; another name is **dynamic dispatch**.

In [ ]:
start: Time = Time(9, 45, 30)
print(start + 3600)

duration: Time = Time(2)
print(start + duration)

<div class="alert alert-success">
    <b>Do It Yourself!</b><br>
    Modify the definition of the &gt; operator for the <i>Time</i> class, so it also compares the Time object against an integer. Use the <i>time_to_int()</i> method to support the comparison.
</div>

In [ ]:
# Remove this line and add your code here

Unfortunately, this implementation of addition is not commutative. If the integer is the
first operand, you get an error message.

In [ ]:
print(3630 + start)

The problem is, instead of asking the `Time` object to add an integer, Python is asking an
integer to add a `Time` object, and it does not know how. 

But there is a clever solution for this problem: the special method `__radd__`, which stands for “right-side add”. 
This method
is invoked when a `Time` object appears on the right side of the `+` operator. 

In [ ]:
class Time:
    """
    Represents the time of day.
    attributes: hour, minute, second
    """
    
    def __init__(self, hour: int = 0, minute: int = 0, second: int = 0) -> None:
        """ 
        Initializes a Time object. It ensures that a valid time is created. 
        That is, 0 <= hour < 24, 0 <= minute < 60, and 0 <= second < 60.
        :param hour: integer value for hour
        :param minute: integer value for minute
        :param second: integer value for second
        """
        (ignore, self.hour) = divmod(hour, 24)
        (ignore, self.minute) = divmod(minute, 60)
        (ignore, self.second) = divmod(second, 60)
            
    def __str__(self) -> str:
        """ 
        Returns a string representation of the Time object.
        :returns: string representation of the Time object.
        """
        return '{:02d}:{:02d}:{:02d}'.format(self.hour, self.minute, self.second)

#...    
    
    def time_to_int(self) -> int:
        """ 
        Converts a Time object into an integer value representing seconds.
        :param time: a Time object to be converted
        :returns: number of seconds as an integer value.
        """
        minutes: int = 60 * self.hour + self.minute
        seconds: int = 60 * minutes + self.second
        return seconds
    
    def increment(self, seconds: int) -> 'Time':
        """ 
        Adds seconds to a Time object.
        :param time: a Time object
        :param seconds: number of seconds to be added
        :returns: a Time object increased by seconds.
        """
        seconds += self.time_to_int()
        return Time.int_to_time(seconds)
    
    def is_after(self, other) -> bool:
        """ 
        Returns a Boolean to indicate whether the current object is greater 
        than another one.
        :param other: another Time object
        :returns: `True` if the current object is greater than time2, 
            `False` otherwise.
        """
        return self.time_to_int() > other.time_to_int()

    def add_time(self, other) -> 'Time':
        """ 
        Adds a Time object to the current Time object.
        :param other: the Time object to be added to the current one
        :returns: a Time object.
        """
        seconds: int = self.time_to_int() + other.time_to_int()
        return Time.int_to_time(seconds)
    
    def __add__(self, other : any) -> 'Time':
        """ 
        Adds a Time object or a number of seconds to the current Time object.
        :param other: the Time object or number of seconds to be added 
            to the current one
        :returns: a Time object.
        """
        if isinstance(other, Time):
            return self.add_time(other)
        else:
            return self.increment(other)
    
    def __radd__(self, other: any) -> 'Time':
        """ 
        Reverse the arguments and adds the Time object or seconds to the 
        current object.
        :param other: the Time object or number of seconds to be added 
            to the current one
        :returns: a Time object.
        """
        return self.__add__(other)
    
    @staticmethod
    def int_to_time(seconds: int) -> 'Time':
        """ 
        Converts an integer value representing seconds into a Time object.
        :param seconds: as Integer value
        :returns: a Time object.
        """
        time: Time = Time()
        minutes, time.second = divmod(seconds, 60)
        hours, time.minute = divmod(minutes, 60)
        _, time.hour = divmod(hours, 24)
    
        return time

In [ ]:
start: Time = Time(9, 45, 30)
print(start + 3600)
print(3600 + start)

duration: Time = Time(2)
print(start + duration)
print(duration + start)

## 7. Polymorphism

Type-based dispatch is useful when it is necessary, but (fortunately) it is not always the case.
Often you can avoid it by writing functions that work correctly for arguments with different types.
Many of the functions we wrote for strings also work for other sequence types. 

In previous examples, we used the `histogram` to count the number of times each letter appears in a word.

In [ ]:
from typing import Dict, List

def histogram(sequence: List[str]) -> Dict[str, int]:
    """ 
    Creates a histogram from a list of elements.
    :param sequence: input list to count number of element occurrences
    :returns: a histogram.
    """
    histogram = dict()
    for elem in sequence:
        if elem not in histogram:
            histogram[elem] = 1
        else:
            histogram[elem] += 1
    return histogram

The function `histogram` works on a list of words.

In [ ]:
topics: List[str] = ['data', 'science', 'statistics', 'machine', 'learning', 'computer', 'science']
histogram(topics)

Or on a list of prime numbers.

In [ ]:
p: List[int] = [3, 7, 11, 13, 3, 5, 23, 13, 3]
histogram(p)

Functions that work with several types are called **polymorphic**.

Polymorphism can facilitate code reuse.
For example, the built-in function `sum`, which adds the elements of a
sequence, works as long as the elements of the sequence support addition.
Since `Time` objects provide an `add` method, they work with `sum`!

In [ ]:
t1: Time = Time(7, 43)
t2: Time = Time(7, 41)
t3: Time = Time(7, 37)

total_time: Time = sum([t1, t2, t3])
print(total_time)

total_value: int = sum([3, 7, 11, 13, 3, 5, 23, 13, 3])
print(total_value)

In general, if all of the operations inside a function work with a given type, the function
works with that type.

<div class="alert alert-success">
    <b>Do It Yourself!</b><br>
    Let's analyse the <i>len()</i> function. Create a list of integers and create string. Make sure the list of integers and the string have different variable names. Then call the function with each one of them. Can we say that the <i>len()</i> function is polymorphic or not?
</div>

In [ ]:
# Remove this line and add your code here

## 8. Interface and Implementation

One of the goals of object-oriented software is to make software more maintainable, which means that you can keep the program working when other parts of the system change, and modify the program to meet new requirements.

A design principle that helps achieve that goal is to keep interfaces separate from implementations. This is called **encapsulation**.

For objects, that means that the methods a class provides should not depend on how the attributes are represented.
For example, in this chapter we developed a class that represents a time of the day. 

Methods provided by this class include `time_to_int`, `is_after`, and `add_time`.
We could implement those methods in several ways. 
The details of the implementation
depend on how we represent time. 

In this chapter, the attributes of a `Time` object are `hour`,
`minute`, and `second`.
As an alternative, we could replace these attributes with a single integer representing the
number of seconds since midnight. 
This implementation would make some methods, like
`is_after`, easier to write, but it makes other methods harder.

After you deploy a new class, you might discover a better implementation. 
If other parts
of the program are using your class, it might be time-consuming and error-prone to change
the interface.
But if you designed the interface carefully, you can change the implementation without
changing the interface, which means that other parts of the program do not have to change.

In [ ]:
start : Time = Time(9, 45, 30)
print(3630 + start)
print(3630 + start + 363)

## 9. Summary

We focussed on more advanced object-oriented features related to methods. We  also showed how to ensure that arguments of methods have correct values. The advanced features allows you to write concise and re-usable code. It also contributes to developing better understandable code. However, it is important to ensure good documentation and tests to guarantee the code quality.

---

# (End of Notebook)